# Behavioral Design Patterns


Behavioral patterns don't really follow any central theme:

- They are all different.
- Sometimes they overlap in their function, i.e., the goal they achieve, but the underlying mechanisms are different.

Table of contents:

- [Chain of Responsibility](#chain-of-responsibility)
  - Example: Support
  - Example: Card Game
  - Example: Broker and Queries
- [Command](#Command)
  - Example: Bank Account with Undo
  - Example: Bank Account with Composite Command = Macro
- [Interpreter](#Interpreter)
- [Iterator](#Iterator)
- [Mediator](#Mediator)
- [Memento](#Memento)
- [Observer](#Observer)
- [State](#State)
- [Strategy](#Strategy)
- [Template](#Template)
- [Visitor](#Visitor)

## Chain of Responsibility

Software components are often built with hierarchical relationships. Those relationships can capture resposibility, similarly as it can happen in a company (CEO, manager, employee). The pattern Chain of Responsibility is a chain of components in which all get a change to process a command or a query, optionally having a default processing implementation and an ability to terminate the processing chain.

In other words, Chain of Responsibility allows an object to pass a request along a chain of potential handlers until the request is handled. This pattern decouples the sender of the request from the receiver by giving multiple objects a chance to handle the request.
 
For example:

- In a GUI, we click a graphical element on a form
  - The button handles the action, which leads to stopping the processing
  - Or the underlying group box can handle it
  - Or the underlying window can handle it
- A card computer game
  - A creature has attack and defense skills
  - Thos skills can be boosted by other card creatures
- Support
  - We can have first-level or basic support
  - Or second-level = advanced support
  - Or third-level = expert support

Some key components that often appear in Chain of Responsibility are:

- **Handler**: Defines an interface for handling requests and for setting the next handler in the chain.
- **ConcreteHandler**: Implements the Handler interface and processes requests it is responsible for. If it can't handle the request, it forwards the request to the next handler in the chain.
- **Client**: Initiates the request and passes it to the first handler in the chain.

**Command and Query Separation (CQS)**: Whenever we operate on objects, we separate all the invokations into two types:

- Command: invokation which asks for an action or a change; e.g., set value to X.
- Query: ask/retrieve information without changing anything, e.g., get value of X.

Separating both Commands and Queries is known as the Command and Query Separation (CQS) Principle. It is a design priciple with some benefits:

- Clarity: Separating commands and queries makes it clear which methods are performing actions and which are retrieving information.
- Simpler Testing: Queries are easier to test because they are idempotent and do not change the state.
- Improved Maintainability: Clear separation of responsibilities makes the code easier to understand and maintain.

### Example: Support Chain

In [7]:
# Abstract Handler:
# Defines an interface for handling requests 
# and for setting the next handler in the chain.
# SupportHandler will be inherited by concrete handlers!
class SupportHandler:
    def __init__(self):
        # The next_handler is a pointer to the next handler
        # which is set with set_next_handler
        # when setting the Chain of Responsibility
        self.next_handler = None

    # This method is used to set the
    # Chain of Responsibility
    def set_next_handler(self, handler):
        self.next_handler = handler

    # This handle_request method is called by the concrete
    # class/object if the type of issue to handle 
    # does not belong to the type of the concrete handler
    def handle_request(self, issue):
        if self.next_handler:
            return self.next_handler.handle_request(issue)
        else:
            return "No handler available for this issue"

In [8]:
# Concrete Handlers:
# They implement the Handler interface 
# and processes requests it is responsible for. 
# If they can't handle the request, 
# they forward the request to the next handler in the chain.
class BasicSupport(SupportHandler):
    def handle_request(self, issue):
        if issue == "basic":
            return "BasicSupport: Handling basic issue"
        else:
            # We call the abstract handler
            # which calls the next handler
            # in the Chain of Responsibility
            return super().handle_request(issue)

class AdvancedSupport(SupportHandler):
    def handle_request(self, issue):
        if issue == "advanced":
            return "AdvancedSupport: Handling advanced issue"
        else:
            return super().handle_request(issue)

class ExpertSupport(SupportHandler):
    def handle_request(self, issue):
        if issue == "expert":
            return "ExpertSupport: Handling expert issue"
        else:
            return super().handle_request(issue)


In [9]:
# __main__: Client
# Initiates the request and passes it 
# to the first handler in the chain.

# Create handlers
basic = BasicSupport()
advanced = AdvancedSupport()
expert = ExpertSupport()

# Set up Chain of Responsibility
# Basic -> Advanced -> Expert
basic.set_next_handler(advanced)
advanced.set_next_handler(expert)

# Client issues
# In this simple example, 
# each issue is a string that specifies the type of issue,
# i.e., we use it to decide which handler should process it
issues = ["basic", "advanced", "expert", "unknown"]

for issue in issues:
    print(f"Issue: {issue} - Response: {basic.handle_request(issue)}")
# Issue: basic - Response: BasicSupport: Handling basic issue
# Issue: advanced - Response: AdvancedSupport: Handling advanced issue
# Issue: expert - Response: ExpertSupport: Handling expert issue
# Issue: unknown - Response: No handler available for this issue


Issue: basic - Response: BasicSupport: Handling basic issue
Issue: advanced - Response: AdvancedSupport: Handling advanced issue
Issue: expert - Response: ExpertSupport: Handling expert issue
Issue: unknown - Response: No handler available for this issue


### Example: Card Chain

Example with card computer game. This example is similar but different to the previous one:

- In the Support example we took and issue and passed it to the Chain of Responsibility until the correct concrete handler is found.
- In this Card Game example, we create a chain of Creature modifiers, i.e., abilities, contained in a root modifier. Then, when we want to use the root, all modifiers are used and each calls to the next in the chain, except when a specific modifier with that function prevents it from propagating.

In [1]:
# Our Card-Creature
class Creature:
    def __init__(self, name, attack, defense):
        self.defense = defense
        self.attack = attack
        self.name = name

    def __str__(self):
        return f'{self.name} ({self.attack}/{self.defense})'

# A Creature-Card can be modified: 
# it collects/looses skills or objects: sword, poison, etc.
# This is where we define our Chain of Responsibility,
# it is our Abstract Modifier.
# We will inherit this base class to Concrete Modifiers.
class CreatureModifier:
    def __init__(self, creature):
        self.creature = creature
        self.next_modifier = None

    def add_modifier(self, modifier):
        if self.next_modifier:
            self.next_modifier.add_modifier(modifier)
        else:
            self.next_modifier = modifier

    # Here is where we apply the Chain of Responsibility
    # This handle() is called from the derived class objects
    def handle(self):
        if self.next_modifier:
            self.next_modifier.handle()

# Concrete Modifier
# Here, we define the concrete handle() methods
class DoubleAttackModifier(CreatureModifier):
    def handle(self):
        print(f'Doubling {self.creature.name}''s attack')
        self.creature.attack *= 2
        super().handle()


class IncreaseDefenseModifier(CreatureModifier):
    def handle(self):
        if self.creature.attack <= 2:
            print(f'Increasing {self.creature.name}''s defense')
            self.creature.defense += 1
        super().handle()

    
class NoBonusesModifier(CreatureModifier):
    def handle(self):
        print('No bonuses for you!')
        # Note that no super().handle() is called
        # That means we don't keep applying the Chain of Responsibility

In [10]:
# __main__
goblin = Creature('Goblin', 1, 1)
print(goblin)

# Abstract Handler: the root
root = CreatureModifier(goblin)

# Chain of Responsibility
root.add_modifier(NoBonusesModifier(goblin))
root.add_modifier(DoubleAttackModifier(goblin))
root.add_modifier(DoubleAttackModifier(goblin))

# Add another modifier - no effect
root.add_modifier(IncreaseDefenseModifier(goblin))

root.handle()  # apply modifiers
print(goblin)

Goblin (1/1)
No bonuses for you!
Goblin (1/1)


### Example: Broker and Queries

In [12]:
from abc import ABC
from enum import Enum

# List of functions we can call
class Event(list):
    def __call__(self, *args, **kwargs):
        for item in self:
            item(*args, **kwargs)


class WhatToQuery(Enum):
    ATTACK = 1
    DEFENSE = 2


class Query:
    def __init__(self, creature_name, what_to_query, default_value):
        self.value = default_value  # bidirectional
        self.what_to_query = what_to_query
        self.creature_name = creature_name

# Broker
class Game:
    def __init__(self):
        self.queries = Event()

    def perform_query(self, sender, query):
        self.queries(sender, query)


class Creature:
    def __init__(self, game, name, attack, defense):
        self.initial_defense = defense
        self.initial_attack = attack
        self.name = name
        self.game = game

    @property
    def attack(self):
        q = Query(self.name, WhatToQuery.ATTACK, self.initial_attack)
        self.game.perform_query(self, q)
        return q.value

    @property
    def defense(self):
        q = Query(self.name, WhatToQuery.DEFENSE, self.initial_attack)
        self.game.perform_query(self, q)
        return q.value

    def __str__(self):
        return f'{self.name} ({self.attack}/{self.defense})'


class CreatureModifier(ABC):
    def __init__(self, game, creature):
        self.creature = creature
        self.game = game
        self.game.queries.append(self.handle)

    def handle(self, sender, query):
        pass

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.game.queries.remove(self.handle)


class DoubleAttackModifier(CreatureModifier):

    def handle(self, sender, query):
        if (sender.name == self.creature.name and
                query.what_to_query == WhatToQuery.ATTACK):
            query.value *= 2


class IncreaseDefenseModifier(CreatureModifier):

    def handle(self, sender, query):
        if (sender.name == self.creature.name and
                query.what_to_query == WhatToQuery.DEFENSE):
            query.value += 3


In [13]:
# __main__
game = Game()
goblin = Creature(game, 'Strong Goblin', 2, 2)
print(goblin)

with DoubleAttackModifier(game, goblin):
    print(goblin)
    with IncreaseDefenseModifier(game, goblin):
        print(goblin)

print(goblin)

Strong Goblin (2/2)
Strong Goblin (4/2)
Strong Goblin (4/5)
Strong Goblin (2/2)


### Command

A Command is an object which represents an instruction to perform a particular action; it contains all the information necessary for the action to be taken.

- Ordinary statements are perishable:
  - They cannot undo member assignment:
  - They cannot directly serialize a sequence of actions (calls).
- We want an object that represents an operation.
  - Example: `person` should change its `age` to value `22`; the operation not only happens, but it is recoded it happened, along with who ordered it.
  - Example: `car` should `start()`.
- There are many uses for Commands: GUI commands, multi-level undo/redo, macro recording, and more!

### Example: Bank Account with Undo

In [18]:
from abc import ABC, abstractmethod
from enum import Enum

# Bank account class
# This class works nicely
# but we'd like to add the undo option
# for the opreations described in it.
# We can do that with a ledger/record.
# We can do that with a Command, too.
class BankAccount:
    OVERDRAFT_LIMIT = -500

    def __init__(self, balance=0):
        self.balance = balance

    def deposit(self, amount):
        self.balance += amount
        print(f'Deposited {amount}, balance = {self.balance}')

    def withdraw(self, amount):
        if self.balance - amount >= BankAccount.OVERDRAFT_LIMIT:
            self.balance -= amount
            print(f'Withdrew {amount}, balance = {self.balance}')
            # We need to return if it succeeded to avoid illegal operations
            return True
        return False

    def __str__(self):
        return f'Balance = {self.balance}'


# Optional, not necessary, but good pratice
class Command(ABC):
    # We could also use __call__, but invoke is more explicit
    @abstractmethod
    def invoke(self):
        pass

    @abstractmethod
    def undo(self):
        pass


# This Command class is a wrapper for the BankAccount
# which adds the Command functionality.
# The action on the BankAccount is not performed instantaneously
# but when we invoke() it.
# Additionally, we can undo the action.
class BankAccountCommand(Command):
    def __init__(self, account, action, amount):
        self.amount = amount
        self.action = action
        self.account = account
        self.success = None

    class Action(Enum):
        DEPOSIT = 0
        WITHDRAW = 1

    def invoke(self):
        if self.action == self.Action.DEPOSIT:
            self.account.deposit(self.amount)
            self.success = True
        elif self.action == self.Action.WITHDRAW:
            self.success = self.account.withdraw(self.amount)

    def undo(self):
        # To avoid illegal operations:
        # if the operation wasn't successful, don't allow undo
        if not self.success:
            return
        # Strictly speaking this is not correct
        # (you don't undo a deposit by withdrawing)
        # but it works for this demo, so...
        if self.action == self.Action.DEPOSIT:
            self.account.withdraw(self.amount)
        elif self.action == self.Action.WITHDRAW:
            self.account.deposit(self.amount)

In [19]:
# __main__
ba = BankAccount()
cmd = BankAccountCommand(ba, BankAccountCommand.Action.DEPOSIT, 100)
cmd.invoke()
print('After $100 deposit:', ba)

cmd.undo()
print('$100 deposit undone:', ba)

illegal_cmd = BankAccountCommand(ba, BankAccountCommand.Action.WITHDRAW, 1000)
illegal_cmd.invoke()
print('After impossible withdrawal:', ba)
illegal_cmd.undo()
print('After undo:', ba)

Deposited 100, balance = 100
After $100 deposit: Balance = 100
Withdrew 100, balance = 0
$100 deposit undone: Balance = 0
After impossible withdrawal: Balance = 0
After undo: Balance = 0


### Example: Bank Account with Composite Command = Macro

This example extends the previous one by adding a Composite construct, which simultaneously represents one Command and a Collection of Commands. That allows transferring money between different bank accounts easily. A Composite Commands is also known as Macro.

In [22]:
import unittest
from abc import ABC, abstractmethod
from enum import Enum


class BankAccount:
    OVERDRAFT_LIMIT = -500

    def __init__(self, balance=0):
        self.balance = balance

    def deposit(self, amount):
        self.balance += amount
        print(f'Deposited {amount}, balance = {self.balance}')

    def withdraw(self, amount):
        if self.balance - amount >= BankAccount.OVERDRAFT_LIMIT:
            self.balance -= amount
            print(f'Withdrew {amount}, balance = {self.balance}')
            return True
        return False

    def __str__(self):
        return f'Balance = {self.balance}'


class Command(ABC):
    def __init__(self):
        self.success = False

    def invoke(self):
        pass

    def undo(self):
        pass


class BankAccountCommand(Command):
    def __init__(self, account, action, amount):
        super().__init__()
        self.amount = amount
        self.action = action
        self.account = account

    class Action(Enum):
        DEPOSIT = 0
        WITHDRAW = 1

    def invoke(self):
        if self.action == self.Action.DEPOSIT:
            self.account.deposit(self.amount)
            self.success = True
        elif self.action == self.Action.WITHDRAW:
            self.success = self.account.withdraw(self.amount)

    def undo(self):
        if not self.success:
            return
        # Strictly speaking this is not correct
        # (you don't undo a deposit by withdrawing)
        # but it works for this demo, so...
        if self.action == self.Action.DEPOSIT:
            self.account.withdraw(self.amount)
        elif self.action == self.Action.WITHDRAW:
            self.account.deposit(self.amount)

In [23]:
# Try using this before using MoneyTransferCommand!
# This is a Composite: both the Command
# and a collection of commands are represented
class CompositeBankAccountCommand(Command, list):
    def __init__(self, items=[]):
        super().__init__()
        for i in items:
            self.append(i)

    def invoke(self):
        for x in self:
            x.invoke()

    def undo(self):
        for x in reversed(self):
            x.undo()


# This class solves the problem from the previous one

class MoneyTransferCommand(CompositeBankAccountCommand):
    def __init__(self, from_acct, to_acct, amount):
        super().__init__([
            BankAccountCommand(from_acct,
                               BankAccountCommand.Action.WITHDRAW,
                               amount),
            BankAccountCommand(to_acct,
                               BankAccountCommand.Action.DEPOSIT,
                               amount)])

    def invoke(self):
        ok = True
        for cmd in self:
            if ok:
                cmd.invoke()
                ok = cmd.success
            else:
                cmd.success = False
        self.success = ok


In [24]:
class TestSuite(unittest.TestCase):
    def test_composite_deposit(self):
        ba = BankAccount()
        deposit1 = BankAccountCommand(ba, BankAccountCommand.Action.DEPOSIT, 1000)
        deposit2 = BankAccountCommand(ba, BankAccountCommand.Action.DEPOSIT, 1000)
        composite = CompositeBankAccountCommand([deposit1, deposit2])
        composite.invoke()
        print(ba)
        composite.undo()
        print(ba)

    def test_transfer_fail(self):
        ba1 = BankAccount(100)
        ba2 = BankAccount()

        # composite isn't so good because of failure
        amount = 1000  # try 1000: no transactions should happen
        wc = BankAccountCommand(ba1, BankAccountCommand.Action.WITHDRAW, amount)
        dc = BankAccountCommand(ba2, BankAccountCommand.Action.DEPOSIT, amount)

        transfer = CompositeBankAccountCommand([wc, dc])

        transfer.invoke()
        print('ba1:', ba1, 'ba2:', ba2)  # end up in incorrect state
        transfer.undo()
        print('ba1:', ba1, 'ba2:', ba2)

    def test_better_tranfer(self):
        ba1 = BankAccount(100)
        ba2 = BankAccount()

        amount = 1000

        transfer = MoneyTransferCommand(ba1, ba2, amount)
        transfer.invoke()
        print('ba1:', ba1, 'ba2:', ba2)
        transfer.undo()
        print('ba1:', ba1, 'ba2:', ba2)
        print(transfer.success)

In [27]:
# __main__
unittest.main(argv=['first-arg-is-ignored'], exit=False)

...

ba1: Balance = 100 ba2: Balance = 0
ba1: Balance = 100 ba2: Balance = 0
False
Deposited 1000, balance = 1000
Deposited 1000, balance = 2000
Balance = 2000
Withdrew 1000, balance = 1000
Withdrew 1000, balance = 0
Balance = 0
Deposited 1000, balance = 1000
ba1: Balance = 100 ba2: Balance = 1000
Withdrew 1000, balance = 0
ba1: Balance = 100 ba2: Balance = 0



----------------------------------------------------------------------
Ran 3 tests in 0.003s

OK


## Interpreter

The Interpreter is related to the processing that text requires to interpret what to do with it:

- Textual input needs to be processed; e.g., code turned in to OOP structures (compilers).
- Some examples:
  - Programming language compilers, interpreters and IDEs.
  - HTML, XML, and similar
  - Numeric expressions (`3 + 4/5`)
  - Regular expressions

Often, the processing is done to obtain structured meaning. To that end:

- We separate it into lexical tokens (*lexing*)
- and then interpret the resulting sequences (*parsing*).

In the following an example numeric expressions with `+` and `-` operations are processed with an interpreter.

### Lexing

In this section, we separate the input string into a sequence of known tokens. That's called *lexing*.

In [32]:
from enum import Enum, auto


class Token:
    # Class-level enum
    class Type(Enum):
        INTEGER = auto()
        PLUS = auto()
        MINUS = auto()
        LPAREN = auto() # (
        RPAREN = auto() # )

    def __init__(self, type, text):
        self.type = type
        self.text = text

    def __str__(self):
        return f'`{self.text}`'


# Lexing
def lex(input):
    result = []

    i = 0
    while i < len(input):
        if input[i] == '+':
            result.append(Token(Token.Type.PLUS, '+'))
        elif input[i] == '-':
            result.append(Token(Token.Type.MINUS, '-'))
        elif input[i] == '(':
            result.append(Token(Token.Type.LPAREN, '('))
        elif input[i] == ')':
            result.append(Token(Token.Type.RPAREN, ')'))
        else:  # must be a number
            digits = [input[i]]
            for j in range(i + 1, len(input)):
                if input[j].isdigit():
                    digits.append(input[j])
                    i += 1
                else:
                    result.append(Token(Token.Type.INTEGER,
                                        ''.join(digits)))
                    break
        i += 1

    return result

### Parsing

In this section, we take the sequence of tokens and interpret their meaning, aka. *parsing*.

In [33]:
class Integer:
    def __init__(self, value):
        self.value = value


class BinaryOperation:
    class Type(Enum):
        ADDITION = 0
        SUBTRACTION = 1

    def __init__(self):
        self.type = None
        self.left = None
        self.right = None

    @property
    def value(self):
        if self.type == self.Type.ADDITION:
            return self.left.value + self.right.value
        elif self.type == self.Type.SUBTRACTION:
            return self.left.value - self.right.value


def parse(tokens):
    result = BinaryOperation()
    have_lhs = False # left-hand-side
    i = 0
    while i < len(tokens):
        token = tokens[i]

        if token.type == Token.Type.INTEGER:
            integer = Integer(int(token.text))
            if not have_lhs:
                result.left = integer
                have_lhs = True
            else:
                result.right = integer
        elif token.type == Token.Type.PLUS:
            result.type = BinaryOperation.Type.ADDITION
        elif token.type == Token.Type.MINUS:
            result.type = BinaryOperation.Type.SUBTRACTION
        elif token.type == Token.Type.LPAREN:  # note: no if for RPAREN
            j = i
            while j < len(tokens):
                if tokens[j].type == Token.Type.RPAREN:
                    break
                j += 1
            # preprocess subexpression
            subexpression = tokens[i + 1:j]
            element = parse(subexpression)
            if not have_lhs:
                result.left = element
                have_lhs = True
            else:
                result.right = element
            i = j  # advance
        i += 1
    return result

In [34]:
def eval(input):
    tokens = lex(input)
    print(' '.join(map(str, tokens)))

    parsed = parse(tokens)
    print(f'{input} = {parsed.value}')

# __main__
eval('(13+4)-(12+1)')
eval('1+(3-4)')

# this won't work
eval('1+2+(3-4)')

`(` `13` `+` `4` `)` `-` `(` `12` `+` `1` `)`
(13+4)-(12+1) = 4
`1` `+` `(` `3` `-` `4` `)`
1+(3-4) = 0
`1` `+` `2` `+` `(` `3` `-` `4` `)`
1+2+(3-4) = 0


## Iterator

- Iteration (traversal) is a core functionality of various data structures; note that some data structures (like trees) don't have a straightforward iteration process.
- An iterator is a class that facilitates the traversal
  - Keeps a reference to the current element.
  - Knows how to move to a different element.
- The iterator protocol requires:
  - `__iter__()` to expose the iterator, which uses
  - `__next__()` to return each of the iterated elements or
  - `raise StopIteration` when it's done.
- In Python, we have the keyword `token` which makes iteration very easy.

In the following, the Iterator of a Binary Tree is shown.

In [35]:
class Node:
    def __init__(self, value, left=None, right=None):
        self.right = right
        self.left = left
        self.value = value

        self.parent = None

        if left:
            self.left.parent = self
        if right:
            self.right.parent = self

    # We expose the tree/Node with __iter__
    def __iter__(self):
        # We use the Iterator we want here
        return InOrderIterator(self)

# Recall we can perform 3 traversals in a tree
#       1
#     /   \
#    2     3
# in-order: 2-1-3 (we implement only this one)
# pre-order: 1-2-3
# post-order: 2-3-1
class InOrderIterator:
    def __init__(self, root):
        self.root = self.current = root
        self.yielded_start = False
        while self.current.left:
            self.current = self.current.left

    # This is a tricky implementation of in-order
    def __next__(self):
        if not self.yielded_start:
            self.yielded_start = True
            return self.current

        if self.current.right:
            self.current = self.current.right
            while self.current.left:
                self.current = self.current.left
            return self.current
        else:
            p = self.current.parent
            while p and self.current == p.right:
                self.current = p
                p = p.parent
            self.current = p
            if self.current:
                return self.current
            else:
              raise StopIteration

def traverse_in_order(root):
    # This is the usual implementation of in-order
    # It leverages the use of yield
    def traverse(current):
        if current.left:
            for left in traverse(current.left):
                yield left
        yield current
        if current.right:
            for right in traverse(current.right):
                yield right
    for node in traverse(root):
        yield node

In [36]:
# __main__

# Recall we can perform 3 traversals in a tree
#       1
#     /   \
#    2     3
# in-order: 2-1-3 (we implement only this one)
# pre-order: 1-2-3
# post-order: 2-3-1
root = Node(1,
            Node(2),
            Node(3))

# We can iterate with iter and next
it = iter(root)
print([next(it).value for x in range(3)])

# Or, since we have __iter__, in a loop
for x in root:
    print(x.value)

for y in traverse_in_order(root):
    print(y.value)

[2, 1, 3]
2
1
3
2
1
3


## Mediator

The Mediator facilitates communication between different components without them necessarily being aware of each other or having direct (reference) access to each other.

- Components may go in and out of a system at any time. For instance:
  - Chatroom participants
  - Players in an online game
- It makes no sense for rthem to have direct reference to one another
  - Because those references may go dead
- Solution: we can have them all refer to some central component that facilitates communication, i.e., the Mediator.

### Example: Chatroom

In [37]:
class Person:
    def __init__(self, name):
        self.name = name
        self.chat_log = []
        self.room = None

    def receive(self, sender, message):
        s = f'{sender}: {message}'
        print(f'[{self.name}\'s chat session] {s}')
        self.chat_log.append(s)

    def say(self, message):
        self.room.broadcast(self.name, message)

    def private_message(self, who, message):
        self.room.message(self.name, who, message)

# This is our Mediator,
# since it allows private and broadcasting messages
class ChatRoom:
    def __init__(self):
        self.people = []

    # Broadcast = message to all
    # We should send a message to everybody
    # except the source
    # We traverse all users and make them receive()
    # the message
    def broadcast(self, source, message):
        for p in self.people:
            if p.name != source:
                p.receive(source, message)

    def join(self, person):
        join_msg = f'{person.name} joins the chat'
        self.broadcast('room', join_msg)
        person.room = self
        self.people.append(person)

    # Message = message to one specififc person
    # We traverse all users and make _only_
    # the specific user receive()
    # the message
    def message(self, source, destination, message):
        for p in self.people:
            if p.name == destination:
                p.receive(source, message)

In [39]:
# __main__
room = ChatRoom()

john = Person('John')
jane = Person('Jane')

room.join(john)
room.join(jane)

john.say('hi room')
jane.say('oh, hey john')

simon = Person('Simon')
room.join(simon)
simon.say('hi everyone!')

jane.private_message('Simon', 'glad you could join us!')

[John's chat session] room: Jane joins the chat
[Jane's chat session] John: hi room
[John's chat session] Jane: oh, hey john
[John's chat session] room: Simon joins the chat
[Jane's chat session] room: Simon joins the chat
[John's chat session] Simon: hi everyone!
[Jane's chat session] Simon: hi everyone!
[Simon's chat session] Jane: glad you could join us!


### Example: Game with Events

In [43]:
class Event(list):
    def __call__(self, *args, **kwargs):
        for item in self:
            item(*args, **kwargs)

# This is our Mediator:
# centrally available component that makes
# possible the communication between entities
class Game:
    def __init__(self):
        self.events = Event()

    def fire(self, args):
        self.events(args)


class GoalScoredInfo:
    def __init__(self, who_scored, goals_scored):
        self.goals_scored = goals_scored
        self.who_scored = who_scored


class Player:
    def __init__(self, name, game):
        self.name = name
        self.game = game
        self.goals_scored = 0

    def score(self):
        self.goals_scored += 1
        args = GoalScoredInfo(self.name, self.goals_scored)
        self.game.fire(args)


class Coach:
    def __init__(self, game):
        game.events.append(self.celebrate_goal)

    def celebrate_goal(self, args):
        if isinstance(args, GoalScoredInfo) and args.goals_scored < 3:
            print(f'Coach says: well done, {args.who_scored}!')


In [44]:
# __main__
game = Game()
player = Player('Sam', game)
coach = Coach(game)

player.score()  # Coach says: well done, Sam!
player.score()  # Coach says: well done, Sam!
player.score()  # ignored by coach

Coach says: well done, Sam!
Coach says: well done, Sam!
